In [2]:
import psycopg2

In [5]:
con = psycopg2.connect(user = "sormusbi",
                       password = "",
                       host = "postgres.keeleressursid.ee",
                       database = "estonian-text-corpora")

try:
    cursor = con.cursor();
except (Exception, psycopg2.Error) as error:
    print("Error in Connection",error)

OperationalError: fe_sendauth: no password supplied


In [3]:
try:
    cursor.execute('SELECT * FROM estonian_text_corpora.koondkorpus_base_v2__v166_morph_analysis__layer LIMIT 10000;')
    texts = cursor.fetchall() 
except (Exception, psycopg2.Error) as error:
    con.rollback()
    print("Error while inserting data into the table:", error)

In [4]:
len(texts)

10000

In [27]:
texts[0][2]['spans'][-1]

{'base_span': [1079, 1080],
 'annotations': [{'form': '',
   'root': '.',
   'lemma': '.',
   'clitic': '',
   'ending': '',
   'root_tokens': ['.'],
   'partofspeech': 'Z',
   'normalized_text': '.'}]}

In [36]:
import time
from datetime import timedelta

In [39]:
print("Start:", str(timedelta(seconds=time.time()+7200)).split(", ")[1])

Start: 22:11:50.724330


In [43]:
start = time.time()
print("Start:", str(timedelta(seconds=start+7200)).split(", ")[1])

paarid = []

t = texts[0][2]['spans']

#for t in analyzed_texts:
#    for sentence in t.sentences:
for text in texts:
    t = text[2]['spans']
    for i, span in enumerate(t):
        annotations = span['annotations']
        #for idx in range(5):
        #    if i + idx < len(sentence):
        if annotations[0]['partofspeech'] != 'S':
            continue

        noun = annotations[0]['lemma']

        if i - 1 >= 0:
            word2 = t[i-1]['annotations'][0]

            if word2['partofspeech'] == 'A':
                paarid.append((word2['lemma'], noun))

        if i - 2 >= 0:
            word3 = t[i-2]['annotations'][0]

            if word3['partofspeech'] == 'A' and t[i-1]['annotations'][0]['partofspeech'] != 'Z':
                paarid.append((word3['lemma'], noun))

        if i - 3 >= 0:
            word4 = t[i-3]['annotations'][0]
            if word4['partofspeech'] == 'A':
                if t[i-1]['annotations'][0]['lemma'] not in ['.', '!', '?'] and t[i-2]['annotations'][0]['lemma'] not in ['.', '!', '?']:
                    paarid.append((word4['lemma'], noun))

        if i + 2 < len(t) - 1:
            if t[i+1]['annotations'][0]['lemma'] == 'olema' and t[i+2]['annotations'][0]['partofspeech'] == 'A':
                paarid.append((t[i+2]['annotations'][0]['lemma'], noun))
                
end = time.time()
print("End:", str(timedelta(seconds=end+7200)).split(", ")[1])
print("Aega kulus:", end-start, "sekundit")

Start: 22:13:08.418729
End: 22:13:16.565456
Aega kulus: 8.146726608276367 sekundit


In [32]:
len(paarid)

116